## Modul Test (Berbasis Proyek)

### Posisi : Machine Vision - AI Engineer

#### Lybrary

!pip install opencv-python
!pip install opencv-python-headless  # Jika Anda tidak memerlukan GUI

In [29]:
import cv2
import numpy as np
import datetime

#### Membaca Video dan Mendeteksi Warna (HSV Color)

In [32]:
cap = cv2.VideoCapture('trafic_light.mp4')

def detect_traffic_light(frame):
    # Konversi ke ruang warna HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definisikan rentang warna untuk merah, kuning, dan hijau
    
    red_lower = np.array([170, 120, 100])
    red_upper = np.array([180, 255, 255])

    yellow_lower = np.array([20, 150, 150])
    yellow_upper = np.array([35, 255, 255])

    green_lower = np.array([35, 100, 100])
    green_upper = np.array([85, 255, 255])

    # Deteksi warna
    red_mask = cv2.inRange(hsv, red_lower, red_upper)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    green_mask = cv2.inRange(hsv, green_lower, green_upper)

    # Lakukan operasi morfologi untuk mengurangi noise
    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)
    yellow_mask = cv2.morphologyEx(yellow_mask, cv2.MORPH_CLOSE, kernel)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_CLOSE, kernel)

    return red_mask, yellow_mask, green_mask

#### Bounding Box, Labeling, Logging the Detections, Image Enhancement

In [33]:
def draw_bounding_box(frame, mask, color_name):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Filter berdasarkan ukuran
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, color_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            log_detection(color_name)

def log_detection(color_name):
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    with open('detection_log.txt', 'a') as log_file:
        log_file.write(f"{timestamp}: {color_name}\n")

def enhance_image(frame):
    # Menajamkan gambar menggunakan kernel
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])  # Kernel penajaman
    enhanced = cv2.filter2D(frame, -1, kernel)
    return enhanced

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Tingkatkan gambar jika diperlukan
    enhanced_frame = enhance_image(frame)

    # Deteksi warna lampu lalu lintas
    red_mask, yellow_mask, green_mask = detect_traffic_light(enhanced_frame)

    # Gambar bounding box untuk setiap warna
    draw_bounding_box(frame, red_mask, 'Red')
    draw_bounding_box(frame, yellow_mask, 'Yellow')
    draw_bounding_box(frame, green_mask, 'Green')

    # Tampilkan frame hasil deteksi
    cv2.imshow('Traffic Light Detection', frame)

    # Keluar jika tombol 'q' ditekan
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Lepaskan objek video dan tutup semua jendela
cap.release()
cv2.destroyAllWindows()